Using following methods to create a AI chatbot:

1. Bag-of-words: It creates indexes of the words.
        For example: "I am Ali" = ["I","am","Ali"]
        I = [1,0,0]
        am = [0,1,0]
        Ali = [0,0,1]
2. Tokenization: It creates an array of every words in a sentence. 
        For example: "I am Ali" = ["I","am","Ali"]
3. Stemming: It creates the root form of the words. Crude heuristic that chops of the end off of words
        For example: "universe","university" = ["univers","univers"]
        
NLP Processing pipeline works this way:

1. Tokenize
2. lower case + stem
3. exclude punctuations and symbols
4. bag of words

In [1]:
import nltk
from nltk.stem.porter import PorterStemmer
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from model import NeuralNet
nltk.download('punkt')

[nltk_data] Downloading package punkt to C:\Users\Ali
[nltk_data]     Mateen\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


True

In [2]:
stemmer = PorterStemmer()
def tokenize(sentence):
    return nltk.word_tokenize(sentence)

def stem(word):
    return stemmer.stem(word.lower())

def bag_of_words(tokenized_sentence, all_words):
    tokenized_sentence = [stem(w) for w in tokenized_sentence]
    bag = np.zeros(len(all_words),dtype=np.float32)
    for idx, w, in enumerate(all_words):
        if w in tokenized_sentence:
            bag[idx] = 1.0
    return bag

This call is just to test the defs
a = "How long does shipping take?"
a = tokenize(a)
print(a)

words = ["Organizer","organizing","organizes"]
stemmed_words = [stem(w) for w in words]
print(stemmed_words)

sentence = ["hello","how","are","you"]
words = ["hi","hello","I","you","bye","thank","cool"]
bag = bag_of_words(sentence, words)
print(bag)

In [4]:
import json
with open('intents.json','r') as f:
    intents = json.load(f)
    
all_words = []
tags = []
xy = []

for intent in intents['intents']:
    tag = intent['tag']
    tags.append(tag)
    for pattern in intent['patterns']:
        w = tokenize(pattern)
        all_words.extend(w)
        xy.append((w, tag))

import string
ignore_words = string.punctuation

all_words = [stem(w) for w in all_words if w not in ignore_words]
all_words = sorted(set(all_words))
tags = sorted(set(tags))

In [5]:
x_train = []
y_train = []

In [7]:
for (pattern_sentence, tag) in xy:
    bag = bag_of_words(pattern_sentence, all_words)
    x_train.append(bag)
    label = tags.index(tag)
    y_train.append(label) #CrossEntropyLoss
    
x_train = np.array(x_train)
y_train = np.array(y_train)

In [27]:
class ChatDataset(Dataset):
    def __init__(self):
        self.n_samples = len(x_train)
        self.x_data = x_train
        self.y_data = y_train
        
    # dataset[idx]
    def __getitem__(self, index):
        return self.x_data[index],self.y_data[index]
    
    def __len__(self):
        return self.n_samples
    
# Hyperparameters
batch_size = 8
hidden_size = 8
output_size = len(tags)
input_size = len(x_train[0])
learning_rate = 0.0001
num_epochs = 10000

dataset = ChatDataset()
train_loader = DataLoader(dataset=dataset, batch_size=batch_size, shuffle=True, num_workers=0)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = NeuralNet(input_size,hidden_size,output_size)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

for epoch in range(num_epochs):
    for (words, labels) in train_loader:
        words = words.to(device)
        labels = labels.to(dtype=torch.long).to(device)
        
        #forward
        outputs = model(words)
        loss = criterion(outputs,labels)
        
        #backward and optimizer step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
    if (epoch +1) % 100 == 0:
        print(f'epoch {epoch+1}/{num_epochs}, loss={loss.item():.4f}')

print(f'Final loss, loss ={loss.item():.4f}')

data = {
    "model_state": model.state_dict(),
    "input_size": input_size,
    "output_size": output_size,
    "hidden_size": hidden_size,
    "all_words": all_words,
    "tags": tags
}

FILE = "data.pth"
torch.save(data, FILE)

print(f'training complete. file saved to {FILE}')

epoch 100/10000, loss=1.7865
epoch 200/10000, loss=2.0055
epoch 300/10000, loss=1.7744
epoch 400/10000, loss=1.6190
epoch 500/10000, loss=1.7201
epoch 600/10000, loss=1.5039
epoch 700/10000, loss=1.2017
epoch 800/10000, loss=0.8141
epoch 900/10000, loss=1.3111
epoch 1000/10000, loss=1.2279
epoch 1100/10000, loss=1.0401
epoch 1200/10000, loss=1.0631
epoch 1300/10000, loss=0.7967
epoch 1400/10000, loss=1.0321
epoch 1500/10000, loss=0.9355
epoch 1600/10000, loss=0.9763
epoch 1700/10000, loss=0.6783
epoch 1800/10000, loss=0.2533
epoch 1900/10000, loss=0.7834
epoch 2000/10000, loss=0.4254
epoch 2100/10000, loss=0.5819
epoch 2200/10000, loss=0.3062
epoch 2300/10000, loss=0.2078
epoch 2400/10000, loss=0.2365
epoch 2500/10000, loss=0.4200
epoch 2600/10000, loss=0.4589
epoch 2700/10000, loss=0.1226
epoch 2800/10000, loss=0.1801
epoch 2900/10000, loss=0.2855
epoch 3000/10000, loss=0.2133
epoch 3100/10000, loss=0.1872
epoch 3200/10000, loss=0.3124
epoch 3300/10000, loss=0.0357
epoch 3400/10000, l